## Library Imports

In [1]:
import pandas as pd
from PyPDF2 import PdfReader as PDFReader, PdfWriter as PDFWriter
import os
import fitz

In [2]:
os.listdir('./')

['.ipynb_checkpoints',
 'BRGY BARITAN.pdf',
 'BRGY TUGATOG.pdf',
 'Created PDFs',
 'Created PDFs 2',
 'EXCEL-ADDRESS, SENIOR SK PWD ILLIT CLASSIFICATION',
 'GENDER W- TOV',
 'municipal_summary.csv',
 'municipal_summary_per_barangay_2023.csv',
 'municipal_summary_per_barangay_2023.xlsx',
 'municipal_summary_per_precinct_2023.csv',
 'municipal_summary_per_precinct_2023.xlsx',
 'PDFS',
 'test.ipynb',
 'test.pdf',
 'test2.ipynb',
 'test3.ipynb',
 'test4.ipynb']

## PDF Segmentation

In [3]:
params = {
    'number': {
        'upper_left': (0, 630),
        'lower_right': (60, 80),
        'upper_right': (60, 630),
        'lower_left': (0, 80),
    },
    'legend_and_voters_name': {
        'upper_left': (60, 630),
        'lower_right': (360, 80),
        'upper_right': (360, 630),
        'lower_left': (60, 80),
    },
    'address': {
        'upper_left': (360, 630),
        'lower_right': (580, 80),
        'upper_right': (580, 630),
        'lower_left': (360, 80),
    }
}

primary_params = {
    'upper_left': (0, 630),
    'lower_right': (60, 80),
    'upper_right': (60, 630),
    'lower_left': (0, 80),
}

ul, ur, ll, lr = [0, 630], [595, 630], [0, 610], [595, 610]

end_line = "We CERTIFY that the above voting records are true, correct and complete."

In [5]:
# checks if the current page contains the end of the list for a certain precinct number
cur = PDFReader('BRGY TUGATOG.pdf')
list_ends = []
for i in range(len(cur.pages)):
    lst = cur.pages[i].extract_text().split('\n')
    check_end_line = False
    for ind in range(len(lst)):
        if lst[ind] == end_line:
            check_end_line = True
            list_ends.append(i)
            break

In [6]:
# assumption here is that the end of the list page with the name and signature texts contains table entries
# if there are no table entries, we basically skip it and move on to the next precinct
cur = PDFReader('BRGY TUGATOG.pdf')
output = PDFWriter()
print(cur.pages[602].extract_text())

BARANGAY : TUGATOG
OCTOBER 30, 2023 BARANGAY AND SANGGUNIANG KABATAAN ELECTIONSRepublic of the Philippines
 
Prec : 0545A
Voter's AddressCITY / MUNICIPALITY : CITY OF MALABON
No. Voter's NameCOMMISSION ON ELECTIONS
LIST OF VOTERS (PRECINCT LEVEL)PROVINCE : NATIONAL CAPITAL REGION - THIRD DISTRICT
PCVL
Name and Signature
(Chairperson)Date Name and Signature
(Chairperson)
ELECTORAL BOARD :We CERTIFY that this list is true and correct and that the same is complete for the precinct.
We CERTIFY that the above voting records are true, correct and complete.
Name and Signature
(Member)ELECTION REGISTRATION BOARD :
DateName and Signature
(Member)Name and Signature
(Member)
Name and Signature
(Member)Representative of Citizens' Arm/Civic Group
Date: Page 7 of  7
8b0d6202308559c0f0cb21eee477c64605-Jun-23 09:33:35.790 PMNOTICE: All authorized recipients of any personal data, personal information, privileged information and sensitive personal information contained in this document, including other 

In [9]:
# test run for brgy tugatog

cur = PDFReader(f'BRGY BARITAN.pdf')
output = PDFWriter()
counter = 0
for i in range(len(cur.pages)):
    page_content = cur.pages[i].extract_text().split('\n')
    check_end, current_index, name_and_signature_found = False, -1, False
    for ind in range(len(page_content)):
        if page_content[ind] == "Name and Signature" and not name_and_signature_found:
            check_end, name_and_signature_found = True, True
            # handle case where the end of the list for a certain precinct is found
            first_number = page_content[8].split()[0]
            if not first_number.isnumeric():
                counter += 1
                # handle the case where the eight index value is not a valid table entry (Name and Signature)
                continue
            else:
                print(cur.pages[i].extract_text())
                # else, find the last number of the last person for that precinct and proceed with cropping
                for j in range(ind-1,-1,-1):
                    if page_content[j].isnumeric():
                        diff = abs(int(first_number) - int(page_content[ind-1].split()[0]))+1
                        break
                
                for key, index in enumerate(params):
                    pg = cur.pages[i]
                    ind = params[index]
                    pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
                    pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
                    pg.cropbox.lower_left = (ind['upper_left'][0], ind['upper_left'][1]-diff*20)
                    pg.cropbox.lower_right = (ind['upper_right'][0], ind['upper_right'][1]-diff*20)
                    output.add_page(pg)
    if not check_end:
        # proceed to standard crop
        for key, index in enumerate(params):
            pg = cur.pages[i]
            ind = params[index]
            pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
            pg.cropbox.lower_right = (ind['lower_right'][0], ind['lower_right'][1])
            pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
            pg.cropbox.lower_left = (ind['lower_left'][0], ind['lower_left'][1])
            output.add_page(pg)
            print(output.extract_text())
        break
            
print(counter + len(output.pages)//3, len(cur.pages))

with open(f'./Created PDFs/{pdfs}.pdf', "wb") as out_f:
    output.write(out_f)

AttributeError: 'PdfWriter' object has no attribute 'extract_text'

In [152]:
for pdfs in os.listdir('./PDFS/'):
    # test run for brgy tugatog
    
    cur = PDFReader(f'./PDFS/{pdfs}')
    output = PDFWriter()
    counter = 0
    for i in range(len(cur.pages)):
        page_content = cur.pages[i].extract_text().split('\n')
        check_end, current_index, name_and_signature_found = False, -1, False
        for ind in range(len(page_content)):
            if page_content[ind] == "Name and Signature" and not name_and_signature_found:
                check_end, name_and_signature_found = True, True
                # handle case where the end of the list for a certain precinct is found
                first_number = page_content[8].split()[0]
                if first_number == 'Name':
                    counter += 1
                    # handle the case where the eight index value is not a valid table entry (Name and Signature)
                    continue
                else:
                    # else, find the last number of the last person for that precinct and proceed with cropping
                    for j in range(ind-1,-1,-1):
                        if page_content[j].isnumeric():
                            diff = abs(int(first_number) - int(page_content[ind-1].split()[0]))+1
                            break
                    
                    for key, index in enumerate(params):
                        pg = cur.pages[i]
                        ind = params[index]
                        pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
                        pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
                        pg.cropbox.lower_left = (ind['upper_left'][0], ind['upper_left'][1]-diff*20)
                        pg.cropbox.lower_right = (ind['upper_right'][0], ind['upper_right'][1]-diff*20)
                        output.add_page(pg)
        if not check_end:
            # proceed to standard crop
            for key, index in enumerate(params):
                pg = cur.pages[i]
                ind = params[index]
                pg.cropbox.upper_left = (ind['upper_left'][0], ind['upper_left'][1])
                pg.cropbox.lower_right = (ind['lower_right'][0], ind['lower_right'][1])
                pg.cropbox.upper_right = (ind['upper_right'][0], ind['upper_right'][1])
                pg.cropbox.lower_left = (ind['lower_left'][0], ind['lower_left'][1])
                output.add_page(pg)
                
    print(counter + len(output.pages)//3, len(cur.pages))
    
    with open(f'./Created PDFs/{pdfs}', "wb") as out_f:
        output.write(out_f)

242 242
411 411
257 257
1124 1124
443 443
380 380
137 137
467 467
291 291
1456 1456
336 336
224 224
196 196
481 481
1243 1243
421 421
410 410
437 437
610 610
1181 1181
625 625


## Parsing

In [161]:
# test parse
pdfs = './Created PDFs/BRGY ACACIA.pdf.pdf'
cur1 = PDFReader(pdfs)
for i in range(0, len(cur1.pages), 3):
    numbers = cur1.pages[i]
    names_and_legends = cur1.pages[i+1]
    addresses = cur1.pages[i+2]
    print(numbers.extract_text())
    break

BARANGAY : ACACIA
OCTOBER 30, 2023 BARANGAY AND SANGGUNIANG KABATAAN ELECTIONSRepublic of the Philippines
 
Prec : 0549A
Voter's AddressCITY / MUNICIPALITY : CITY OF MALABON
No. Voter's NameCOMMISSION ON ELECTIONS
LIST OF VOTERS (PRECINCT LEVEL)PROVINCE : NATIONAL CAPITAL REGION - THIRD DISTRICT
PCVL
1 62 SISA ST.  ACACIA  MALABON CITY ABAD, CARLITO CANONIZADOC
2 62 SISA ST.  ACACIA  MALABON CITY ABAD, JENNIFER FLORES
3 106 SISA ST.  ACACIA  MALABON CITY ALEJANDRIA, RIA CASTILLO
4 98 E BASILIO ST. ALERA, REGINE SELGA*
5 25 BASILIO APALIS, ARIZZA SUELEN
6 108-3 DAANGBAKAL. ACACIA  MALABON CITY AQUINO, JOSEPH LAZAGA
7 162 SISA ST. AQUINO, KIEL MANALANG*
8 108-3 RAILROAD ST.  ACACIA  MALABON CITY AQUINO, ROWELL BUSALPA
9 119 CONSUELO ARCADIO, CHRISTENSEN JOYCE GUIGUINTO*
10 153 SISA ST. ARTAJO, RUBY CANOPIN
11 14 CONSUELO ST AUSTRIA, JULIE IGNACIO
12 07 CONSUELO ST BAISA, TRISHA ANNE BASA*
13 146 DAANG BAKAL ST., BANIQUED, ROLANDO TORIO
14 10 FLERIDA ST BAUTISTA, JOSE CONRAD VICTORIANO
15

In [ ]:
import numpy as np

dct = {}
for pdfs in os.listdir('./Created PDFs'):
    cur = PDFReader(f'./Created PDFs/{pdfs}')
    document = fitz.open(f'./Created PDFs/{pdfs}')
    
    for i in range(0, len(cur.pages), 3):
        another_dct = {}
        precinct_number = cur.pages[i].extract_text().split('\n')[3]
        # print(precinct_number[-5:])
        page1 = document.load_page(i).get_text()
        page1 = page1.split('\n')
        page2 = document.load_page(i+1).get_text()
        page2 = page2.split('\n')
        page1, page2 = page1[:-1], page2[:-1]
        index = 0
        # print(precinct_number, page1)
        for ind in range(len(page1)):
            if precinct_number[-5:] not in dct:
                dct[precinct_number[-5:]] = {}
            dct[precinct_number[-5:]][page1[ind]] = {'name': page2[index], 'age_bracket': (page2[index+1] if index+1 < len(page2) and (page2[index+1] == 'C' or page2[index+1] == '*') else np.nan)}
            index += 1 + (index + 1 < len(page2) and (page2[index+1] == 'C' or page2[index+1] == '*'))
    break

Prec : 0549A ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27']
Prec : 0549A ['28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54']
Prec : 0549A ['55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81']
Prec : 0549A ['82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108']
Prec : 0549A ['109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135']
Prec : 0549A ['136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '1

In [ ]:
print(dct)

In [29]:
dct['0549A']['169']

KeyError: '0549A'

## Second Segmentation

In [60]:
pdf = './PDFS/BRGY BARITAN.pdf'
cur = PDFReader(pdf)
output = PDFWriter()
checked = False
for i in range(8):    
    ul, ur, ll, lr = [0, 630], [595, 630], [0, 615], [595, 615]
    for j in range(27):
        pg = cur.pages[i]
        # ctx = pg.extract_text().split('\n') # list that contains the length of
        pg.cropbox.upper_left = ul
        pg.cropbox.upper_right = ur
        pg.cropbox.lower_left = ll
        pg.cropbox.lower_right = lr
        ul[1], ur[1], ll[1], lr[1] = ul[1]-20, ur[1]-20, ll[1]-20, lr[1]-20
        if j == 2 or j == 23:
            ul[1], ur[1], ll[1], lr[1] = ul[1], ur[1], ll[1]-10, lr[1]-10
            checked = True
        elif checked:
            ul[1], ur[1], ll[1], lr[1] = ul[1]-5, ur[1]-5, ll[1], lr[1]
            checked = False
        # print(ul, ur, ll, lr)
        output.add_page(pg)
    # break
with open("test.pdf", "wb") as out_f:
    output.write(out_f)

In [5]:
# take the number of rows an entry takes up in each page
pdf = './PDFS/BRGY BARITAN.pdf'
cur = PDFReader(pdf)
output = PDFWriter()
dct = {}
for i in range(len(cur.pages)):
    lst = []
    pg = cur.pages[i]
    ctx = pg.extract_text().split('\n')
    prec_number = ctx[3][-5:]
    if prec_number not in dct:
        dct[prec_number] = 0
    j = 8
    while True:
        cur_str = ctx[j].split()
        check = False
        for k in range(len(cur_str)):
            if cur_str[k] == 'Name' or cur_str[k] == 'Date:':
                check = True
                break
        if check:
            break
        next_str = ctx[j+1].split()
        if not next_str[0].isnumeric() and next_str[0] != 'Date:':
            lst.append(2)
            j += 1
        else:
            lst.append(1)
        j += 1
    dct[prec_number] += len(lst)
    if prec_number == '0129A':
        print(len(lst))
        # print(pg.extract_text())
for key, value in dct.items():
    print(key, value)

25
25
23
26
27
25
25
19
0
0126A 198
0126B 193
0127A 195
0127B 181
0128A 191
0129A 195
0129B 194
0130A 186
0131A 193
0132A 199
0133A 182
0134A 192
0135A 183
0135B 191
0135C 177
0135D 198
0136A 188
0136B 187
0136C 186
0136D 137
0137A 189
0137B 195
0138A 193
0139A 178
0140A 184
0140B 198
0141A 183
0141B 182
0142A 189
0142B 194
0143A 183
0143B 184
0144A 193
0145A 194
0146A 197
0147A 192
0147B 189
0147C 194
0147D 193
0148A 200
0148B 199
0148C 200
0149A 195
0150A 199
0151A 200
0152A 187
0152B 189
0152C 180
0153A 187
0154A 190


In [82]:
# test parse
pdfs = './PDFS/BRGY BARITAN.pdf'
cur1 = PDFReader(pdfs)
pgs = cur1.pages[16].extract_text()
print(pgs.split('\n'))

['BARANGAY : BARITAN', 'OCTOBER 30, 2023 BARANGAY AND SANGGUNIANG KABATAAN ELECTIONSRepublic of the Philippines', ' ', 'Prec : 0126B', "Voter's AddressCITY / MUNICIPALITY : CITY OF MALABON", "No. Voter's NameCOMMISSION ON ELECTIONS", 'LIST OF VOTERS (PRECINCT LEVEL)PROVINCE : NATIONAL CAPITAL REGION - THIRD DISTRICT', 'PCVL', '190 4 STA. ANA BARITAN VILLEGAS, MARIA KATRINA GALAURAN*', '191 60 B. STA ANA ST STO ROSARIO ST VILLEGAS, VANN MARLO MATUTINA*', '192 19 TALABAHAN YARRA, CARLOTA SULIT', '193 19 TALABAHAN YARRA, JEFFREY JIONGCO', 'Name and Signature', '(Chairperson)Date Name and Signature', '(Chairperson)', 'ELECTORAL BOARD :We CERTIFY that this list is true and correct and that the same is complete for the precinct.', 'We CERTIFY that the above voting records are true, correct and complete.', 'Name and Signature', '(Member)ELECTION REGISTRATION BOARD :', 'DateName and Signature', '(Member)Name and Signature', '(Member)', 'Name and Signature', "(Member)Representative of Citizens'

In [7]:
df = pd.read_csv('municipal_summary_per_precinct_2023.csv')
df.head()

,Unnamed: 0,Precinct,Registered Voters Total,Registered Voters Male,Registered Voters Female,Voters Who Actually Voted (Total),Voters Who Actually Voted (Male),Voters Who Actually Voted (Female),Voting Percentage (Total),Voting Percentage (Male),Voting Percentage (Female),Barangay
0,0,0126A,198,84,114,194,82,112,97.98,97.62,98.25,BRGY BARITAN
1,1,0126B,193,89,104,186,83,103,96.37,93.26,99.04,BRGY BARITAN
2,2,0127A,195,86,109,184,79,105,94.36,91.86,96.33,BRGY BARITAN
3,3,0127B,181,87,94,1,0,1,0.55,0.00,1.06,BRGY BARITAN
4,4,0128A,191,83,108,169,72,97,88.48,86.75,89.81,BRGY BARITAN


In [8]:
ans = 0
cnt = 0
for key, value in dct.items():
    cnt += 1
    print(type(df[df['Precinct'] == key]['Registered Voters Total']), int(df[df['Precinct'] == key]['Registered Voters Total']), value)
    ans += (int(df[df['Precinct'] == key]['Registered Voters Total']) == value)
    if int(df[df['Precinct'] == key]['Registered Voters Total']) != value:
        print(key)
print(ans, cnt)

<class 'pandas.core.series.Series'> 198 198
<class 'pandas.core.series.Series'> 193 193
<class 'pandas.core.series.Series'> 195 195
<class 'pandas.core.series.Series'> 181 181
<class 'pandas.core.series.Series'> 191 191
<class 'pandas.core.series.Series'> 194 195
0129A
<class 'pandas.core.series.Series'> 193 194
0129B
<class 'pandas.core.series.Series'> 186 186
<class 'pandas.core.series.Series'> 192 193
0131A
<class 'pandas.core.series.Series'> 197 199
0132A
<class 'pandas.core.series.Series'> 182 182
<class 'pandas.core.series.Series'> 191 192
0134A
<class 'pandas.core.series.Series'> 183 183
<class 'pandas.core.series.Series'> 191 191
<class 'pandas.core.series.Series'> 177 177
<class 'pandas.core.series.Series'> 198 198
<class 'pandas.core.series.Series'> 188 188
<class 'pandas.core.series.Series'> 186 187
0136B
<class 'pandas.core.series.Series'> 186 186
<class 'pandas.core.series.Series'> 137 137
<class 'pandas.core.series.Series'> 188 189
0137A
<class 'pandas.core.series.Series'

C:\Users\user\AppData\Local\Temp\ipykernel_22504\1668539966.py:5: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(type(df[df['Precinct'] == key]['Registered Voters Total']), int(df[df['Precinct'] == key]['Registered Voters Total']), value)
C:\Users\user\AppData\Local\Temp\ipykernel_22504\1668539966.py:6: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  ans += (int(df[df['Precinct'] == key]['Registered Voters Total']) == value)
C:\Users\user\AppData\Local\Temp\ipykernel_22504\1668539966.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  if int(df[df['Precinct'] == key]['Registered Voters Total']) != value:


In [10]:
string = "We CERTIFY that this list is true and correct and that the same is complete for the precinct. We CERTIFY that the above voting records are true, correct and complete. Name and Signature (Member) ELECTION REGISTRATION BOARD : Representative of Citizens' Arm/Civic Group ELECTORAL BOARD Date"
string = string.split()
string

['We',
 'CERTIFY',
 'that',
 'this',
 'list',
 'is',
 'true',
 'and',
 'correct',
 'and',
 'that',
 'the',
 'same',
 'is',
 'complete',
 'for',
 'the',
 'precinct.',
 'We',
 'CERTIFY',
 'that',
 'the',
 'above',
 'voting',
 'records',
 'are',
 'true,',
 'correct',
 'and',
 'complete.',
 'Name',
 'and',
 'Signature',
 '(Member)',
 'ELECTION',
 'REGISTRATION',
 'BOARD',
 ':',
 'Representative',
 'of',
 "Citizens'",
 'Arm/Civic',
 'Group',
 'ELECTORAL',
 'BOARD',
 'Date']

In [11]:
dct = {}
for pdfs in os.listdir('./PDFS'):
    cur = PDFReader(f'./PDFS/{pdfs}')
    output = PDFWriter()
    for i in range(len(cur.pages)):
        lst = []
        pg = cur.pages[i]
        ctx = pg.extract_text().split('\n')
        prec_number = ctx[3][-5:]
        if prec_number not in dct:
            dct[prec_number] = []
        j = 8
        while True:
            cur_str = ctx[j].split()
            check = False
            for k in range(len(cur_str)):
                if cur_str[k] in string:
                    check = True
                    break
            if check:
                break
            next_str = ctx[j+1].split()
            if not next_str[0].isnumeric():
                lst.append(2)
                j += 1
            else:
                lst.append(1)
            j += 1
        dct[prec_number].append(lst)

In [16]:
validate = {}
for pdf in os.listdir('./PDFS/'):
    cur = PDFReader(f'./PDFS/{pdf}')
    output = PDFWriter()
    checked = False
    for i in range(len(cur.pages)):
        prec_number = cur.pages[i].extract_text().split('\n')[3][-5:]
        ul, ur, ll, lr = [0, 630], [595, 630], [0, 615], [595, 615]
        if prec_number not in validate:
            validate[prec_number] = 0
        print(prec_number, (dct[prec_number])[validate[prec_number]], validate[prec_number])
        print((dct[prec_number])[validate[prec_number]])
        for j in range(len((dct[prec_number])[validate[prec_number]])):
            pg = cur.pages[i]
            pg.cropbox.upper_left = ul
            pg.cropbox.upper_right = ur
            pg.cropbox.lower_left = ll
            pg.cropbox.lower_right = lr
            ul[1], ur[1], ll[1], lr[1] = ul[1]-20, ur[1]-20, ll[1]-20, lr[1]-20
            if ((dct[prec_number])[validate[prec_number]])[j] == 2:
                ul[1], ur[1], ll[1], lr[1] = ul[1], ur[1], ll[1]-10, lr[1]-10
                checked = True
            elif checked:
                ul[1], ur[1], ll[1], lr[1] = ul[1]-5, ur[1]-5, ll[1], lr[1]
                checked = False
            output.add_page(pg)
        validate[prec_number] += 1
    break
    with open(f'./Created PDFs 2/{pdf}', "wb") as out_f:
        output.write(out_f)

0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 0
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 3
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 4
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
0549A [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2] 5
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [45]:
xparams = {
    'number': (0, 60),
    'name_and_class': (60, 360),
    'address': (360, 595)
}

In [57]:
validate = {}
for pdf in os.listdir('./PDFS/'):
    cur = PDFReader(f'./PDFS/{pdf}')
    output = PDFWriter()
    checked = False
    for i in range(len(cur.pages)):
        prec_number = cur.pages[i].extract_text().split('\n')[3][-5:]
        ul, ur, ll, lr = [0, 630], [595, 630], [0, 615], [595, 615]
        if prec_number not in validate:
            validate[prec_number] = 0
        # print(prec_number, (dct[prec_number])[validate[prec_number]], validate[prec_number])
        # print((dct[prec_number])[validate[prec_number]])
        for j in range(len((dct[prec_number])[validate[prec_number]])):
            for key, index in xparams.items():
                ul[0], ur[0], ll[0], lr[0] = index[0], index[1], index[0], index[1]
                pg = cur.pages[i]
                pg.cropbox.upper_left = ul
                pg.cropbox.upper_right = ur
                pg.cropbox.lower_left = ll
                pg.cropbox.lower_right = lr
                output.add_page(pg)
                # print(ul, ur, ll, lr)
            # update part
            ul[1], ur[1], ll[1], lr[1] = ul[1]-20, ur[1]-20, ll[1]-20, lr[1]-20
            if j+1 < len(((dct[prec_number])[validate[prec_number]])) and ((dct[prec_number])[validate[prec_number]])[j+1] == 2:
                if checked:
                    ul[1], ur[1], ll[1], lr[1] = ul[1]-6, ur[1]-6, ll[1]-6, lr[1]-6
                else:
                    ul[1], ur[1], ll[1], lr[1] = ul[1], ur[1], ll[1]-10, lr[1]-10
                    checked = True
            elif checked:
                ul[1], ur[1], ll[1], lr[1] = ul[1]-5, ur[1]-5, ll[1], lr[1]
                checked = False
        validate[prec_number] += 1
    with open(f'./Created PDFs 2/{pdf}', "wb") as out_f:
        output.write(out_f)

## Read PDF contents

In [97]:
import re

def is_valid_combination(s):
    # Define the pattern for valid combinations
    pattern = r'^[ABC\*]{4}$'
    pattern1 = r'^[ABC\*]{1}$'
    pattern2 = r'^[ABC\*]{2}$'
    pattern3 = r'^[ABC\*]{3}$'
    # Check if the string matches the pattern
    if re.match(pattern, s) or re.match(pattern1, s) or re.match(pattern2, s) or re.match(pattern3, s):
        # Check if each character appears only once
        if len(set(s)) == len(s):
            return True
        else:
            return False
    else:
        return False

In [98]:
is_valid_combination('C')

True

In [114]:
import csv
for pdfs in os.listdir('./Created PDFs 2'):
    lst = []
    nlst = []
    cur = PDFReader(f'./Created PDFs 2/{pdfs}')
    with fitz.open(f'./Created PDFs 2/{pdfs}') as pdf_document:
        # Iterate through each page
        for page_num in range(0, len(pdf_document), 3):
            qlst = []
            prec_number = cur.pages[page_num].extract_text().split('\n')[3][-5:]
            # Extract text from the page
            for j in range(3):
                page = pdf_document.load_page(page_num+j)
                qlst.append(page.get_text())
            qlst.append(prec_number)
            lst.append(qlst)
    for i in range(len(lst)):
        if lst[i][1][:-1].split('\n')[0][0] == ',':
            lst[i][1] = lst[i][1][2:]
        if len(lst[i][1][:-1].split('\n')) == 2 and is_valid_combination(lst[i][1][:-1].split('\n')[1]):
            lst[i] = [lst[i][0][:-1]] + lst[i][1][:-1].split('\n') + [lst[i][2][:-1].replace('\n', ' '), lst[i][3]]
        else:
            lst[i] = [lst[i][0][:-1], lst[i][1][:-1], '', lst[i][2][:-1].replace('\n', ' '), lst[i][3]]
    j = 0
    while j < len(lst):
        if j+1 < len(lst) and lst[j][0] == lst[j+1][0]:
            nlst.append([lst[j][0], lst[j][1], lst[j][2] + " " + lst[j+1][2], lst[i][3], pdfs[:-4]])
            j += 1
        else:
            nlst.append(lst[j] + [pdfs[:-4]])
        j += 1
    filename = f'./CSVs/{pdfs[:-4]}.csv'
    # Write the data to a CSV file
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(nlst)
    print(f'{pdfs} done.')

BRGY ACACIA.pdf done.
BRGY BARITAN.pdf done.
BRGY BAYAN BAYANAN.pdf done.
BRGY CATMON.pdf done.
BRGY CONCEPCION  NAME AND PREC.pdf done.
BRGY DAMPALIT.pdf done.
BRGY FLORES.pdf done.
BRGY HULONG DUHAT.pdf done.
BRGY IBABA.pdf done.
BRGY LONGOS.pdf done.
BRGY MAYSILO.pdf done.
BRGY MUZON.pdf done.
BRGY NIUGAN.pdf done.
BRGY PANGHULO.pdf done.
BRGY POTRERO.pdf done.
BRGY SAN AGUSTIN.pdf done.
BRGY SANTOLAN.pdf done.
BRGY TANONG.pdf done.
BRGY TINAJEROS.pdf done.
BRGY TONSUYA.pdf done.
BRGY TUGATOG.pdf done.


In [72]:
for i in range(len(lst)):
    lst[i] = [lst[i][0][:-1], lst[i][1][:-1], lst[i][2][:-1]]
for i in range(5):
    print(lst[i])

['1', 'ABUYO, EDNA NIEDO\nC', '36 STA. ANA ST., STO. ROSARIO']
['2', 'ALABAN, ALISON DARIO', '25 STA. ANA STO. ROSARIO']
['3', 'ALCANTARA, RICARDO PALOMPO', '112 ILANG-ILANG ST']
['4', 'ALMARIO, ANITA SAMSON\nC', '99 STA ANA EXT.  BARITAN  MALABON METRO\nMAL.']
['5', 'ALMARIO, MARIA ANNA SAMSON', '99 STA. ANA EXT. STO. ROSARIO, BARITAN']


In [85]:
import csv
filename = './data.csv'

# Write the data to a CSV file
with open(filename, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(nlst)

In [86]:
import pandas as pd

df = pd.read_csv('./data.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 391: invalid continuation byte